In [112]:
%run /home/jovyan/data_loading_tools.ipynb
%run /home/jovyan/plotting_tools.ipynb

In [113]:
# Client täytyy käynnistää ekana
client = start_client()

In [114]:
# Alustetaan datan hakua
parameters = ['Temperature']
time_range = 10                   # Määrittää kuinka monelta päivältä dataa haetaan tästä hetkestä eteenpäin
model = 'EC'

delayed_data_ec = get_model_data_delay(parameters, time_range, model)
EC_params = merge_model_delayed(client, delayed_data_ec)

Done!


In [115]:
EC_params.head().compute()

<xarray.Dataset>
Dimensions:            (time: 5, y: 5, x: 5)
Coordinates:
  * time               (time) datetime64[ns] 2024-09-11 ... 2024-09-11T04:00:00
  * y                  (y) float32 -3.883e+06 -3.876e+06 ... -3.854e+06
  * x                  (x) float32 -1.01e+06 -1.003e+06 ... -9.806e+05
    lat                (y, x) float32 52.7 52.72 52.73 ... 52.98 53.0 53.01
    lon                (y, x) float32 5.421 5.523 5.625 ... 5.519 5.621 5.724
Data variables:
    crs                int16 -32767
    air_temperature_4  (time, y, x) float32 284.3 284.7 285.7 ... 283.9 284.3
Attributes:
    Conventions:  CF-1.6
    title:        <title>
    institution:  fmi.fi
    source:       <producer>

In [116]:
# # Otetaan data ulos xarrayna, koska vain yksi käyttäjä DASK-järjestelmässä ja serverin käyttö hankaloittaa tekemistä porukalla
T2m = EC_params['air_temperature_4'].compute()
lat = EC_params['lat'].compute()
lon = EC_params['lon'].compute()
time = EC_params['time'].compute()

In [117]:
# Satunnaista testailua lokaalilla

T_max = (T2m.max(dim="time").compute())
#print(T_max.values)
T_hour = (T2m.groupby("time.hour").mean().compute())
#print(T_hour.values)
T_days = (T2m.resample(time="1D"))
#print(T_days)

#print(T2m.isel(time=5).values)


# Tästä eteenpäin tehdään visualisointeja

In [118]:
# Alla oleva tapa on "vanha" eli plotataan data lokaalilla alustalla.

In [119]:
def draw(parameter, t): 
    # Määritellään kartta:
    ax = generate_fig_properties(lat, lon)

    # Käsitellään dataa
    chart = T2m.isel(time=t)
    chart = chart - 273.15
    cmap = 'nipy_spectral'
    vmin = -40
    vmax = 40
    
    pcolormesh_map_plt(chart, vmin, vmax, cmap, lon, lat)

In [120]:
params = ['2 metre temperature']
widgets.interact(draw, parameter=params, t=(0, len(time) - 1))

interactive(children=(Dropdown(description='parameter', options=('Floating 5 hour average temperature',), valu…

<function __main__.draw(parameter, t)>